In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd 
data_train = pd.read_csv('/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Malyalam/Mal_sentiment_full_train.tsv', sep = '\t')
data_val = pd.read_csv('/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Malyalam/Mal_sentiment_full_dev.tsv', sep = '\t')
data_test = pd.read_csv('/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Malyalam/Mal_sentiment_full_test_withoutlabels.tsv', sep = '\t')

In [3]:
from sklearn import preprocessing
import numpy as np

le = preprocessing.LabelEncoder()

le.fit(data_train['category'])
print(le.classes_)

y_train_index = le.transform(data_train['category'])
y_val_index = le.transform(data_val['category'])



['Mixed_feelings' 'Negative' 'Positive' 'not-malayalam' 'unknown_state']


In [4]:
from keras.utils.np_utils import to_categorical
y_train_cal  =  to_categorical(y_train_index)
y_val_cal  =  to_categorical(y_val_index)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvec = TfidfVectorizer(ngram_range=(1,6), analyzer = 'char_wb', max_features = 30000)
#tfidfvec = TfidfVectorizer(ngram_range=(1,3), max_features = 30000)
tfidf_train = tfidfvec.fit_transform(data_train['text'])
tfidf_val = tfidfvec.transform(data_val['text'])

In [6]:
x_train = tfidf_train
x_test = tfidf_val
y_train = y_train_cal
y_test =y_val_cal

In [7]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout
model = Sequential()
model.add(Dense(4096, input_dim = tfidf_train.shape[1], activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4096)              122884096 
_________________________________________________________________
dropout (Dropout)            (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2097664   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 125,014,917
Trainable params: 125,014,917
Non-trainable params: 0
__________________________________________

In [8]:
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(x_train.toarray(), y_train, validation_data=(x_test.toarray(), y_test), epochs=20, batch_size=20, verbose=2 )

Epoch 1/20
795/795 - 35s - loss: 0.2937 - accuracy: 0.6716 - val_loss: 0.2532 - val_accuracy: 0.7214
Epoch 2/20
795/795 - 20s - loss: 0.1717 - accuracy: 0.8287 - val_loss: 0.2754 - val_accuracy: 0.7231
Epoch 3/20
795/795 - 20s - loss: 0.0840 - accuracy: 0.9248 - val_loss: 0.3374 - val_accuracy: 0.7231
Epoch 4/20
795/795 - 20s - loss: 0.0357 - accuracy: 0.9688 - val_loss: 0.3658 - val_accuracy: 0.7288
Epoch 5/20
795/795 - 20s - loss: 0.0210 - accuracy: 0.9812 - val_loss: 0.4646 - val_accuracy: 0.7225
Epoch 6/20
795/795 - 20s - loss: 0.0146 - accuracy: 0.9837 - val_loss: 0.5585 - val_accuracy: 0.7339
Epoch 7/20
795/795 - 20s - loss: 0.0120 - accuracy: 0.9851 - val_loss: 0.6111 - val_accuracy: 0.7356
Epoch 8/20
795/795 - 20s - loss: 0.0114 - accuracy: 0.9863 - val_loss: 0.6618 - val_accuracy: 0.7378
Epoch 9/20
795/795 - 20s - loss: 0.0142 - accuracy: 0.9840 - val_loss: 0.5660 - val_accuracy: 0.7208
Epoch 10/20
795/795 - 20s - loss: 0.0137 - accuracy: 0.9848 - val_loss: 0.5783 - val_accura

In [10]:
tfidf_test = tfidfvec.transform(data_test['text'])

In [11]:
predictions_test_submission = model.predict(tfidf_test.toarray())

In [12]:
predictions_test_submission

array([[2.7502561e-04, 1.5857559e-05, 1.1249874e-05, 5.5392698e-06,
        9.9969232e-01],
       [7.8509438e-05, 2.2441338e-04, 2.0023771e-02, 1.4363950e-03,
        9.7823685e-01],
       [6.0310978e-10, 1.7766313e-10, 6.3966021e-10, 2.0931958e-10,
        1.0000000e+00],
       ...,
       [1.4620035e-07, 9.9998951e-01, 4.7951448e-06, 1.2587533e-09,
        5.5656860e-06],
       [4.3541797e-09, 9.5102960e-01, 8.5500865e-08, 6.0595688e-11,
        4.8970278e-02],
       [6.8120055e-02, 1.7130680e-05, 9.3185925e-01, 6.1150494e-08,
        3.4626396e-06]], dtype=float32)

In [13]:
import numpy as np
predictions_test_submission = np.asarray(predictions_test_submission)

In [14]:
import numpy as np
predictions_test_submission1 = np.zeros_like(predictions_test_submission)
predictions_test_submission1[np.arange(len(predictions_test_submission)), predictions_test_submission.argmax(1)] = 1

In [15]:
predictions_test_submission1

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [16]:
predictions_test_submission1_final = np.argmax(predictions_test_submission1, axis=1)

In [17]:
predictions_test_submission1_final1 = le.inverse_transform(predictions_test_submission1_final)

In [18]:
predictions_test_submission1_final1

array(['unknown_state', 'unknown_state', 'unknown_state', ..., 'Negative',
       'Negative', 'Positive'], dtype=object)

In [19]:
data_test['category'] = predictions_test_submission1_final1

In [20]:
data_test

,id,text,category
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,unknown_state
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,unknown_state
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,unknown_state
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive
...,...,...,...
1957,Mal_1958,Expression വച്ചു നോക്കുമ്പോൾ മൂക്കുത്തി കട്ടത്...,unknown_state
1958,Mal_1959,എനിക്ക് ഒരു 100 like തരുമോ,unknown_state
1959,Mal_1960,Nannayi onnu poliyunna lakshnm unde...Thrill m...,Negative
1960,Mal_1961,Athikam eduthu kayatti vekkenda....direction k...,Negative


In [21]:
data_test.to_csv("/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Malyalam/DNN_Malayalam_72.tsv", sep="\t", index=False)

In [22]:
check_submission = pd.read_csv("/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Malyalam/DNN_Malayalam_72.tsv", sep="\t")

In [23]:
check_submission

,id,text,category
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,unknown_state
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,unknown_state
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,unknown_state
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive
...,...,...,...
1957,Mal_1958,Expression വച്ചു നോക്കുമ്പോൾ മൂക്കുത്തി കട്ടത്...,unknown_state
1958,Mal_1959,എനിക്ക് ഒരു 100 like തരുമോ,unknown_state
1959,Mal_1960,Nannayi onnu poliyunna lakshnm unde...Thrill m...,Negative
1960,Mal_1961,Athikam eduthu kayatti vekkenda....direction k...,Negative
